Running with Bunnies
====================

You and your rescued bunny prisoners need to get out of this collapsing death trap of a space station - and fast! Unfortunately, some of the bunnies have been weakened by their long imprisonment and can't run very fast. Their friends are trying to help them, but this escape would go a lot faster if you also pitched in. The defensive bulkhead doors have begun to close, and if you don't make it through in time, you'll be trapped! You need to grab as many bunnies as you can and get through the bulkheads before they close. 

The time it takes to move from your starting point to all of the bunnies and to the bulkhead will be given to you in a square matrix of integers. Each row will tell you the time it takes to get to the start, first bunny, second bunny, ..., last bunny, and the bulkhead in that order. The order of the rows follows the same pattern (start, each bunny, bulkhead). The bunnies can jump into your arms, so picking them up is instantaneous, and arriving at the bulkhead at the same time as it seals still allows for a successful, if dramatic, escape. (Don't worry, any bunnies you don't pick up will be able to escape with you since they no longer have to carry the ones you did pick up.) You can revisit different spots if you wish, and moving to the bulkhead doesn't mean you have to immediately leave - you can move to and from the bulkhead to pick up additional bunnies if time permits.

In addition to spending time traveling between bunnies, some paths interact with the space station's security checkpoints and add time back to the clock. Adding time to the clock will delay the closing of the bulkhead doors, and if the time goes back up to 0 or a positive number after the doors have already closed, it triggers the bulkhead to reopen. Therefore, it might be possible to walk in a circle and keep gaining time: that is, each time a path is traversed, the same amount of time is used or added.

Write a function of the form solution(times, time_limit) to calculate the most bunnies you can pick up and which bunnies they are, while still escaping through the bulkhead before the doors close for good. If there are multiple sets of bunnies of the same size, return the set of bunnies with the lowest prisoner IDs (as indexes) in sorted order. The bunnies are represented as a sorted list by prisoner ID, with the first bunny being 0. There are at most 5 bunnies, and time_limit is a non-negative integer that is at most 999.

For instance, in the case of
[
  [0, 2, 2, 2, -1],  # 0 = Start
  [9, 0, 2, 2, -1],  # 1 = Bunny 0
  [9, 3, 0, 2, -1],  # 2 = Bunny 1
  [9, 3, 2, 0, -1],  # 3 = Bunny 2
  [9, 3, 2, 2,  0],  # 4 = Bulkhead
]
and a time limit of 1, the five inner array rows designate the starting point, bunny 0, bunny 1, bunny 2, and the bulkhead door exit respectively. You could take the path:

   Start    End   Delta Time Status
    -   0     -    1 Bulkhead initially open
    0   4    -1    2
    4   2     2    0
    2   4    -1    1
    4   3     2   -1 Bulkhead closes
    3   4    -1    0 Bulkhead reopens; you and the bunnies exit

With this solution, you would pick up bunnies 1 and 2. This is the best combination for this space station hallway, so the answer is [1, 2].

Languages
=========

To provide a Java solution, edit Solution.java
To provide a Python solution, edit solution.py

Test cases
==========
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Java cases --

Input:

Solution.solution({{0, 1, 1, 1, 1}, {1, 0, 1, 1, 1}, {1, 1, 0, 1, 1}, {1, 1, 1, 0, 1}, {1, 1, 1, 1, 0}}, 3)

Output:
    [0, 1]

Input:

Solution.solution({{0, 2, 2, 2, -1}, {9, 0, 2, 2, -1}, {9, 3, 0, 2, -1}, {9, 3, 2, 0, -1}, {9, 3, 2, 2, 0}}, 1)

Output:
    [1, 2]

-- Python cases --

Input:

solution.solution([[0, 2, 2, 2, -1], [9, 0, 2, 2, -1], [9, 3, 0, 2, -1], [9, 3, 2, 0, -1], [9, 3, 2, 2, 0]], 1)

Output:
    [1, 2]

Input:

solution.solution([[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 3)

Output:
    [0, 1]

In [3]:
import itertools

In [10]:
def solution(times, time_limit):
    # The algorithm I propose is kind of hard coding. I didn't outsourced any available TSM algorithm.
    # Instead, I have managed to tackle the problem with my own logic.
    
    number_of_bunnies = len(times)-2
    leng = len(times)

    perms2List = list(itertools.permutations(range(leng),2))
    perms2Durations = dict.fromkeys(perms2List)
    
    # Find the shortest distance between any 2 points. I will consider 3 cases. First one is the direct path. Second one is putting
    # a single node between them. Last one is putting 2 nodes between them. 
    for key in perms2Durations.keys():
        
        remaining_nodes = [node for node in range(leng) if (node != key[0] and node!= key[1])]
        dist = times[key[0]][key[1]] # go directly
        
        # put single node in-between:
        for node in remaining_nodes:
            dist = min(dist, times[key[0]][node] + times[node][key[1]])
        # put 2 nodes in-between:
        for node in list(itertools.permutations(remaining_nodes,2)):
            dist = min(dist, times[key[0]][node[0]] + times[node[0]][node[1]] + times[node[1]][key[1]]) 
        perms2Durations[key] = dist

    # Check if there exists a loop with minus time change in total or a negative entry on the diagonal of times.
    # If there exists such one, return the list of all bunnies immediately. 
    for ends in range(leng):
        if (times[ends][ends] < 0): # Check if there is aa negative entry on the diagonal of times. 
            return [bun for bun in range(number_of_bunnies)]
        
        for mid in [i for i in range(leng) if i != ends]: # Check if there is a loop with minus time change in total.
            if perms2Durations[(ends,mid)]+perms2Durations[(mid,ends)] < 0:
                return [bun for bun in range(number_of_bunnies)]
    
    # Go from maximum number of bunnies to save down to to 1
    for max_bunnies_saved in range(number_of_bunnies, 0, -1):
        permsList = list(itertools.permutations( range( 1,leng-1 ),max_bunnies_saved ))
        permsDurations = dict.fromkeys(permsList)
    
        # Find the duration for all possible routes with the same number of bunnies saved. 
        # If you find a duration less than time_limit, return the bunny list
        for key in sorted(permsDurations.keys()):
            duration = perms2Durations[(0,key[0])] + perms2Durations[(key[-1],leng-1)]
            
            for keyception in range(max_bunnies_saved-1):
                duration += perms2Durations[(key[keyception],key[keyception+1])]
            if duration <= time_limit:
                return sorted([bun-1 for bun in key])
            
    return []

In [11]:
times = [[-1, 2, 2, 2, -1],
         [9, 0, 2, 2, -1],
         [9, 3, 0, 2, -1],
         [9, 3, 2, 0, -1],
         [9, 3, 2, 2, 0]]
time_limit = 0 # [1,2] should be rescued. (provided case)

times2 = [[0, 1, 1, 1, 1], 
          [1, 0, 1, 1, 1], 
          [1, 1, 0, 1, 1], 
          [1, 1, 1, 0, 1], 
          [1, 1, 1, 1, 0]]
time_limit2 = 3 # [0,1] should be rescued. (provided case)

times3=  [[1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1]]
time_limit3 = 1 # [] should be rescued.

times4 = [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]
time_limit4 = 2 # [0] should be rescued.

times5 = [[0, 5, 11, 11, 1],
         [10, 0, 1, 5, 1],
         [10, 1, 0, 4, 0],
         [10, 1, 5, 0, 1],
         [10, 10, 10, 10, 0]]
time_limit5 = 10 # [0,1] should be rescued.

times6=[[0, 2, 2, 2, -1],
        [9, 0, 2, 2, 0],
        [9, 3, 0, 2, 0],
        [9, 3, 2, 0, 0],
        [-1, 3, 2, 2, 0]]
time_limit6 = -500 # [0,1,2] should be rescued.

times7 =[[0, 10, 10, 10, 1],
        [0, 0, 10, 10, 10],
        [0, 10, 0, 10, 10],
        [0, 10, 10, 0, 10],
        [1, 1, 1, 1, 0]]
time_limit7 = 5 # [0,1] should be rescued.

times8=[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]
time_limit8 = 1 # [0,1,2] should be rescued.

times9=[[2, 2],
        [2, 2]]
time_limit9 = 1 # [] should be rescued.

times10=[[0, 10, 10, 1, 10],
        [10, 0, 10, 10, 1],
        [10, 1, 0, 10, 10],
        [10, 10, 1, 0, 10],
        [1, 10, 10, 10, 0]]
time_limit10 = 6 # [0,1,2] should be rescued.

times11 =[[0,  1,  5,  5,  2],
          [10, 0,  2,  6,  10],
          [10, 10, 0,  1,  5],
          [10, 10, 10, 0,  1],
          [10, 10, 10, 10, 0]]
time_limit11 = 5 # [0, 1, 2] should be rescued.

times12 =[[0, 1, 3, 4, 2],
          [10, 0, 2, 3, 4],
          [10, 10, 0, 1, 2],
          [10, 10, 10, 0, 1],
          [10, 10, 10, 10, 0]]
time_limit12 = 4 # [] should be rescued.

times13 =[[0, 2, 2, 2, -1],
          [9, 0, 2, 2, -1],
          [9, 3, 0, 2, -1],
          [9, 3, 2, 0, -1],
          [9, 3, 2, 2, 0]]
time_limit13 = 1 # [1,2] should be rescued.

times14 =[[0,  1, 10, 10, 10],
          [10, 0,  1,  1,  2],
          [10, 1,  0, 10, 10],
          [10, 1,  10, 0, 10],
          [10, 10, 10, 10, 0]]
time_limit14 = 7 # [0,1,2] should be rescued.

times15 =[[0, 5, 11, 11, 1],
          [10, 0, 1, 5, 1],
          [10, 1, 0, 4, 0],
          [10, 1, 5, 0, 1],
          [10, 10, 10, 10, 0]]
time_limit15 = 10 # [0,1] should be rescued.

times16 =[[0, 20, 20, 20, -1],
          [90, 0, 20, 20, 0],
          [90, 30, 0, 20, 0],
          [90, 30, 20, 0, 0],
          [-1, 30, 20, 20, 0]]
time_limit16 = 0 # [0,1,2] should be rescued.

times17 =[[0, 10, 10, 10, 1],
          [0, 0, 10, 10, 10],
          [0, 10, 0, 10, 10],
          [0, 10, 10, 0, 10],
          [1, 1, 1, 1, 0]]
time_limit17 = 5 # [0,1] should be rescued.

times18 =[[0, 10, 10, 1, 10],
          [10, 0, 10, 10, 1],
          [10, 1, 0, 10, 10],
          [10, 10, 1, 0, 10],
          [1, 10, 10, 10, 0]]
time_limit18 = 6 # [0,1,2] should be rescued.

times19 =[[1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1]]
time_limit19 = 1 # [] should be rescued.

times20 =[[0, 0, 1, 1, 1],
          [0, 0, 0, 1, 1],
          [0, 0, 0, 0, 1],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]
time_limit20 = 0 # [0,1,2] should be rescued.

times21 =[[1, 1, 1, 1, 1],
          [-1, 1, 1, 1, 1],
          [-1, 1, 1, 1, 1],
          [-1, 1, 1, 1, 1],
          [-1, 1, 1, 1, 1]]
time_limit21 = 1 # [0, 1, 2] should be rescued.

times22 =[[0, 1, 5, 5, 5, 5],
          [5, 0, 1, 5, 5, 5],
          [5, 5, 0, 5, 5, -1],
          [5, 5, 1, 0, 5, 5],
          [5, 5, 1, 5, 0, 5],
          [5, 5, 1, 1, 1, 0]]
time_limit22 = 3 # [0,1,2,3] should be rescued.

In [12]:
solution(times22, time_limit22)

[0, 1, 2, 3]